In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

from autogen_core.models import ModelFamily
model_client = OpenAIChatCompletionClient(
    model="qwen2.5:7b",
    # model="llama3.2:3b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.ANY,
    },
)

assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=model_client,
)

# Use asyncio.run(...) when running in a script.
response = await assistant_agent.on_messages(
    [TextMessage(content="Write a 3 line poem on lake tangayika", source="user")], CancellationToken()
)
print(response.chat_message.content)

Along Tanganyika's shore,  
Blue depths hide ancient secrets,  
Nature's lens, wild and wore.


In [2]:
agent_state = await assistant_agent.save_state()
print(agent_state)

{'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a 3 line poem on lake tangayika', 'source': 'user', 'type': 'UserMessage'}, {'content': "Along Tanganyika's shore,  \nBlue depths hide ancient secrets,  \nNature's lens, wild and wore.", 'thought': None, 'source': 'assistant_agent', 'type': 'AssistantMessage'}]}}


In [3]:
new_assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=model_client,
)
await new_assistant_agent.load_state(agent_state)

# Use asyncio.run(...) when running in a script.
response = await new_assistant_agent.on_messages(
    [TextMessage(content="What was the last line of the previous poem you wrote", source="user")], CancellationToken()
)
print(response.chat_message.content)

The last line of the previous poem is: "Nature's lens, wild and wore."


In [4]:
# Define a team.
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=model_client,
)
agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))

# Run the team and stream messages to the console.
stream = agent_team.run_stream(task="Write a beautiful poem 3-line about lake tangayika")

# Use asyncio.run(...) when running in a script.
await Console(stream, output_stats=True)

# Save the state of the agent team.
team_state = await agent_team.save_state()

---------- user ----------
Write a beautiful poem 3-line about lake tangayika
---------- assistant_agent ----------
Amidst the mountains' embrace,  
Lake Tanganyika dozes in grace,  
Mirrors the sky in emerald pace.
[Prompt tokens: 30, Completion tokens: 29]
---------- Summary ----------
Number of messages: 2
Finish reason: Maximum number of messages 2 reached, current message count: 2
Total prompt tokens: 30
Total completion tokens: 29
Duration: 1.65 seconds


In [5]:
print(team_state)

{'type': 'TeamState', 'version': '1.0.0', 'agent_states': {'assistant_agent': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a beautiful poem 3-line about lake tangayika', 'source': 'user', 'type': 'UserMessage'}, {'content': "Amidst the mountains' embrace,  \nLake Tanganyika dozes in grace,  \nMirrors the sky in emerald pace.", 'thought': None, 'source': 'assistant_agent', 'type': 'AssistantMessage'}]}}, 'message_buffer': []}, 'RoundRobinGroupChatManager': {'type': 'RoundRobinManagerState', 'version': '1.0.0', 'message_thread': [{'source': 'user', 'models_usage': None, 'metadata': {}, 'content': 'Write a beautiful poem 3-line about lake tangayika', 'type': 'TextMessage'}, {'source': 'assistant_agent', 'models_usage': {'prompt_tokens': 30, 'completion_tokens': 29}, 'metadata': {}, 'content': "Amidst the mountains' embrace,  \nLake Tanganyika dozes in grace,  \nMir

In [7]:
await agent_team.reset()
stream = agent_team.run_stream(task="What was the last line of the poem you wrote?")
await Console(stream, output_stats=True)

---------- user ----------
What was the last line of the poem you wrote?
---------- assistant_agent ----------
As I didn't write a specific poem that we've been working on together, I don't have a particular last line to share. Could you provide more context or tell me if you'd like me to compose a new poem for you? That way, I can ensure it meets your expectations!
[Prompt tokens: 29, Completion tokens: 60]
---------- Summary ----------
Number of messages: 2
Finish reason: Maximum number of messages 2 reached, current message count: 2
Total prompt tokens: 29
Total completion tokens: 60
Duration: 2.96 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last line of the poem you wrote?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=60), metadata={}, content="As I didn't write a specific poem that we've been working on together, I don't have a particular last line to share. Could you provide more context or tell me if you'd like me to compose a new poem for you? That way, I can ensure it meets your expectations!", type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

In [8]:
# Load team state.
await agent_team.load_state(team_state)
stream = agent_team.run_stream(task="What was the last line of the poem you wrote?")
await Console(stream, output_stats=True)

---------- user ----------
What was the last line of the poem you wrote?
---------- assistant_agent ----------
The last line of the poem is: "Mirrors the sky in emerald pace."
[Prompt tokens: 79, Completion tokens: 19]
---------- Summary ----------
Number of messages: 2
Finish reason: Maximum number of messages 2 reached, current message count: 2
Total prompt tokens: 79
Total completion tokens: 19
Duration: 1.43 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last line of the poem you wrote?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=79, completion_tokens=19), metadata={}, content='The last line of the poem is: "Mirrors the sky in emerald pace."', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

In [11]:
import json

## save state to disk

with open("data/team_state.json", "w") as f:
    json.dump(team_state, f)

## load state from disk
with open("data/team_state.json", "r") as f:
    team_state = json.load(f)

new_agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))
await new_agent_team.load_state(team_state)
stream = new_agent_team.run_stream(task="What was the last line of the poem you wrote?")
await Console(stream, output_stats=True)

---------- user ----------
What was the last line of the poem you wrote?
---------- assistant_agent ----------
The last line of the poem is: "Mirrors the sky in emerald pace."
[Prompt tokens: 79, Completion tokens: 19]
---------- Summary ----------
Number of messages: 2
Finish reason: Maximum number of messages 2 reached, current message count: 2
Total prompt tokens: 79
Total completion tokens: 19
Duration: 1.06 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last line of the poem you wrote?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=79, completion_tokens=19), metadata={}, content='The last line of the poem is: "Mirrors the sky in emerald pace."', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')